In [1]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace


In [2]:
# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [3]:
# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

DEBUG:lib.lab_lib:hello


# TODO

1. ✅CG, MGで時間をExtra-Pから取得できることを確認
2. ✅時間の単位を「秒」にする
    * 列「Inclusive total msec」に一部存在している”：”で区切られた値の単位は、msec ではなく分と秒を区切る”：”
3. ✅相対コストの算出関数の実装
4. ✅Extra-P のDocker環境の構築
4. Extra-P による予測との組み合わせを行う
    * なにをどう組み合わせるのかがわかっていないので、それは確かめる
    * 元データ, Extra-P単体で予測したデータ, <何か> で結果を作成

# 予測結果に必要なもの

* ✅元データ（予測環境の生データ）
* ✅Extra-P単体で予測したデータ
* Extra-Pで各関数の実行時間を予測し、それにコール回数を掛けた値で予測したデータ
    * コール回数で実行時間を割る
    * Extra-Pでモデルを作成 <- コール回数で実行時間を割る
    * 👆を利用して予測
    * 本プログラムでモデルを作成 <- コール回数で実行時間を割る
    * 👆を利用して予測
    * 「コール回数 * 実行時間」を計算して予測

上記のセルよりモデルは次式

$$ 2.5521930556315375 + 0.013737041505459383 * iteration + 3.41165137739477 * 10^7 * iteration * size^3 * \log_2 size $$

In [4]:
train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48, 64]

test_lulesh_processes: list[int] = [512]
test_lulesh_iterations: list[int] = [256]
test_lulesh_sizes: list[int] = [128]


In [5]:
trainDF_lulesh :pd.DataFrame = return_rawDF_lulesh(
    list_process = train_lulesh_processes,
    list_iteration = train_lulesh_iterations,
    list_size = train_lulesh_sizes,
    csvDir=csvDirPath
)

In [6]:
trainDF_lulesh["InclusivePerCall"] = -1
trainDF_lulesh = trainDF_lulesh.reset_index()
for i, sr in trainDF_lulesh.iterrows():
    trainDF_lulesh.at[i, "InclusivePerCall"] = convertPprofTime(sr["Inclusive"]) / float(sr["#Call"])
trainDF_lulesh

,index,%Time,Exclusive,Inclusive,#Call,#Subrs,Name,process,iteration,size,InclusivePerCall
0,0,100.0,0.00387,"6,478",1.000000,1.000,.TAU_application,8,8,16,6.478000e+00
1,1,100.0,25,"6,478",1.000000,100.125,int_main(int_char_**),8,8,16,6.478000e+00
2,2,87.4,"5,663","5,663",1.000000,0.000,MPI_Finalize(),8,8,16,5.663000e+00
3,3,11.9,768,768,1.000000,0.000,MPI_Init(),8,8,16,7.680000e-01
4,4,0.2,6,11,8.000000,32768.000,void_CalcKinematicsForElems(Domain,8,8,16,1.375000e-03
...,...,...,...,...,...,...,...,...,...,...,...
4795,27,0.0,0.00875,0.00875,1.000000,0.000,void_Domain::SetupSymmetryPlanes(Int_t),343,128,64,8.750000e-06
4796,28,0.0,0.00158,0.00346,1.000000,2.000,void_ParseCommandLineOptions(int_char_**_Int_t...,343,128,64,3.460000e-06
4797,29,0.0,0.00188,0.00188,2.000000,0.000,StrToInt,343,128,64,9.400000e-07
4798,30,0.0,0.000443,0.000443,1.000000,0.000,MPI_Comm_size(),343,128,64,4.430000e-07


In [ ]:
%reset

***

---

___

# 2022年4月17日～

次のような表を作成する

採用される割合 (MAPE の最大値 [%] ，MAPE の最小値 [%]) [%]

| ベンチマークプログラム名 | 線形モデル               | 対数モデル               | 反比例モデル              |
|--------------|---------------------|---------------------|---------------------|
| str          | float(float, float) | float(float, float) | float(float, float) |


目標となるのは一気にこのベンチマークプログラムを作成することだが、既存のライブラリ関数などを利用し、まずはベンチマークごとに作成可能にする。

メモ

## 実装予定

1. 行方向に最小値を検出
2. 最小値以外をNaNに変更
3. 列方向に最小値と最大値を検出

## 


***

---

___